In [2]:
import torch

In [3]:
weights = torch.load('/fhome/gia07/project/runs/run5/weights/CNN_autoencoder_epoch_14.pth')


In [10]:
import torch

# Load the model weights
model_weights = torch.load('/fhome/gia07/project/runs/run5/weights/CNN_autoencoder_epoch_14.pth')

# Count the number of weights
total_weights = 0
for param in model_weights.values():
    num_weights = param.numel()
    total_weights += num_weights

print("Total number of weights:", total_weights)

Total number of weights: 1678659


In [13]:
from utils import load_model

UNET_NOT_RESIDUALS = load_model('UNET_NOT_RESIDUALS')
UNET_NOT_RESIDUALS.load_state_dict(model_weights)

<All keys matched successfully>

Inference Time Autoncoder

In [27]:
import numpy as np
model = UNET_NOT_RESIDUALS
device = torch.device("cuda")
model.to(device)
dummy_input = torch.randn(1, 3,225,225, dtype=torch.float).to(device)

# INIT LOGGERS
starter, ender = torch.cuda.Event(enable_timing=True), torch.cuda.Event(enable_timing=True)
repetitions = 300
timings=np.zeros((repetitions,1))
#GPU-WARM-UP
for _ in range(100):
    _ = model(dummy_input)
# MEASURE PERFORMANCE
with torch.no_grad():
    for rep in range(repetitions):
        starter.record()
        _ = model(dummy_input)
        ender.record()
        # WAIT FOR GPU SYNC
        torch.cuda.synchronize()
        curr_time = starter.elapsed_time(ender)
        timings[rep] = curr_time

mean_syn = np.sum(timings) / repetitions
std_syn = np.std(timings)
print(round(mean_syn, 3))

3.106


In [37]:
weights_effcient = torch.load('/fhome/gia07/project/runs_clf/efficientnet-2c/weights/x/CNN_autoencoder_epoch_5.pth')
efficient_net = load_model('EFFICIENTNET', 2)


In [38]:
efficient_net.load_state_dict(weights_effcient)

<All keys matched successfully>

In [40]:
import numpy as np
model = efficient_net
device = torch.device("cuda")
model.to(device)
dummy_input = torch.randn(1, 3,225,225, dtype=torch.float).to(device)

# INIT LOGGERS
starter, ender = torch.cuda.Event(enable_timing=True), torch.cuda.Event(enable_timing=True)
repetitions = 300
timings=np.zeros((repetitions,1))
#GPU-WARM-UP
for _ in range(100):
    _ = model(dummy_input)
# MEASURE PERFORMANCE
with torch.no_grad():
    for rep in range(repetitions):
        starter.record()
        _ = model(dummy_input)
        ender.record()
        # WAIT FOR GPU SYNC
        torch.cuda.synchronize()
        curr_time = starter.elapsed_time(ender)
        timings[rep] = curr_time

mean_syn = np.sum(timings) / repetitions
std_syn = np.std(timings)
print(round(mean_syn, 3))

47.22
